
# GENERACIÓN DE INFORMES CLÍNICOS UTILIZANDO LA API DE OPENAI
El código que aparece en este documento se utiliza para generar informes clínicos en distintos idiomas. Por cómo está diseñado el código, se pueden generar los informes clínicos que sean necesarios.
Para obtener los informes clínicos, se utiliza la API de openAI. Los informes posteriormente se guardan en un JSON (que se utiliza para poder hacer NER y para las pruebas), y en un dichero .docx, para que sean legibles.


Instalemos su api...

In [ ]:
!pip install -q openai

Se importa su api:

In [ ]:
from openai import OpenAI

Para poder utilizar variables de entorno

In [ ]:
from google.colab import userdata


Generación de informes clínicos

In [ ]:
def generate_reports(language, country, required_entity, number_of_reports=5):
  client = OpenAI(api_key=userdata.get('apiKey'))

  system_content_text = f"You are a doctor from {country}. You are writing typical clinical reports in {language}."

  user_content_text = f"Write {number_of_reports} clinical reports. Mark its endindg with the string \"----\". You must include: {required_entity}."
  completion =   client.chat.completions.create(
      model="gpt-4o",
      messages=[
            {
              "role": "system",
              "content": [
                {
                  "type": "text",
                  "text": system_content_text
                }
              ]
            },

          {"role": "user", "content": user_content_text}

      ]
  )
  return completion.choices[0].message.content


Función para añadir los informes a una única lista reports:

In [ ]:
def extract_reports_into_list(completion, reports):
  # Access the content (string) of the message using .content
  for s in completion.split("----"):
    if(len(s) > 0 and s != "  " ):
      reports.append(s)

Función para crear un JSON a partir de una lista de reports

In [ ]:
  import json

In [ ]:
def create_json_from_list(reports):
  dictionary = {}
  for i in range(len(reports)):
    dictionary[f"Report{i}"] = reports[i]

  with open('data.json', 'w') as f:
   json.dump(dictionary, f, indent=4)


Funciones para crear los informes clínicos:

In [ ]:
def create_reports_from_one_demand(reports,language, country, required_entity, number_of_reports=5):
  completion = generate_reports(language, country, required_entity, number_of_reports)
  extract_reports_into_list(completion, reports)
def create_reports_from_demands(demands):
  reports = []
  for language, country, required_entity ,number_of_reports in demands:
    create_reports_from_one_demand(reports, language, country, required_entity, number_of_reports)
  create_json_from_list(reports)


Se utilizan las funciones anteriores para crear los informes clínicos que se utilizarán para los tests.

In [ ]:
testRequirements = "4 names of patients, 4 ages, 4 times any sex, 4 heights, 4 weights, 5 diagnosis and medical procedures, 5 diseases, 5 admittance and discharge dates, 5 medicines and their dosages (units and values). But you do not need to mention explicitely on the report the words disease, diagnosis... Write the clinical report as a doctor would."
reqPruebas = "4 nombres de pacientes, 4 edades, 4 veces algún sexo, 4 alturas, 4 pesos, 5 diagnósticos y procedimientos médicos, 5 enfermedades, 5 fechas de ingreso y de alta, 5 fármacos con sus medidas (unidades y valor). Pero no necesitas mencionar explítamente en el informe las palabras enfermedad, diagnóstico...Escribe el informe como lo haría un médico"
demands = [("English", "England", testRequirements,4 ), ("spanish", "Spain", reqPruebas ,4)]
create_reports_from_demands(demands)




["Certainly, I can help prepare concise clinical reports for a variety of clinical case scenarios. Here are five reports, each carefully detailing a different case:\n\n**Clinical Report 1: Case of Hypertension**\n\nPatient Name: John Smith  \nAge: 58  \nGender: Male  \nDate of Visit: 12th October 2023  \nReason for Visit: Routine Checkup\n\n**History:**  \nJohn Smith, a 58-year-old male, presented to the clinic for a routine checkup. He reported no significant symptoms of concern but mentioned occasional headaches, which he attributed to stress. His medical history reveals a diagnosis of hypertension five years prior, for which he is on amlodipine 5 mg once daily. He also reported adhering to a moderately healthy lifestyle, although he admitted to a sedentary job and occasional fatigue following physical activity.\n\n**Physical Examination:**  \nUpon physical examination, Mr. Smith appeared generally well, with a body weight of 95 kg and height of 1.75 meters, indicating a BMI of 31.0 

Se utizan las funciones anteriores para crear informes más largos

In [ ]:
testRequirements = "A thousand words per report"
reqPruebas = "1000 palabras por informe"
demands = [("English", "England", testRequirements,5 ), ("Spanish", "Spain", reqPruebas ,5)]
create_reports_from_demands(demands)

Se utilizan las funciones anteriores para crear 150 informes cínicos. 50 en inglés, 50 en español y 50 en francés.

In [ ]:
demands = [("English", "England", testRequirements,10), ("English", "England", testRequirements,10), ("English", "England", testRequirements,10), ("English", "England", testRequirements,10), ("English", "England", testRequirements,10),("spanish", "Spain", reqPruebas ,10), ("spanish", "Spain", reqPruebas ,10),("spanish", "Spain", reqPruebas ,10), ("spanish", "Spain", reqPruebas ,10),("spanish", "Spain", reqPruebas ,10),("French", "France", testRequirements,10), ("French", "France", testRequirements,10), ("French", "France", testRequirements,10), ("French", "France", testRequirements,10), ("French", "France", testRequirements,10)]

create_reports_from_demands(demands)


#Conversión a .docx

Se convierten los resultados a un dichero .docx para aumentar su legibilidad


In [ ]:
!pip install -q python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
from docx import Document
def convert2docxOneReport(report_text, doc):
  paragraphs = report_text.split("\n")

  for paragraph in paragraphs:
      bold_text_split = paragraph.split("**")
      paragraph_adder = doc.add_paragraph()
      for i, part in enumerate(bold_text_split):
          if i % 2 == 0:
              # Regular text
              paragraph_adder.add_run(part)
          else:
              # Bold text
              bold_run = paragraph_adder.add_run(part)
              bold_run.bold = True
  doc.add_page_break()


def convert2docx(list_of_reports):
  #Creating the document
  doc = Document()
  for i,report in enumerate(list_of_reports):
    doc.add_heading(f"Report{i}", level=1)
    convert2docxOneReport(report, doc)
  doc.save("Reports.docx")




Provisional
*texto en cursiva*

In [ ]:
with(open("data.json")) as f:
  data = json.load(f)
reports = []
for key in data:
  reports.append(data[key])
convert2docx(reports)